In [ ]:

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']


video = cv2.VideoCapture(path_vid)

    trackerleft = cv2.TrackerCSRT_create()
    trackerright = cv2.TrackerCSRT_create()

if not video.isOpened():

    print("Could not open video")

    sys.exit()

ok, frame = video.read()

if not ok:
    print('Cannot read video file')
    sys.exit()

# Define an initial bounding box
bbox = (287, 23, 86, 320)

# Uncomment the line below to select a different bounding box
bbox = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
trackerleft.init(frame, bbox)
trackerright.init(frame, bbox)



while True:
    # Read a new frame
    ok, frame = video.read()
    if not ok:
        break
        
    # Update tracker
    ok, bbox = tracker.update(frame)
    
    # Draw bounding box
    if ok:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
    cv2.imshow("Tracking", frame)

    if cv2.waitKey(1) & 0xff == 27 : 
        break
        
cv2.destroyAllWindows()
video.release()

In [ ]:
fvs = FileVideoStream(path_vid).start()
frame = fvs.read()
frame = cv2.flip(frame, 1)

trackerleft = cv2.TrackerCSRT_create()
trackerright = cv2.TrackerCSRT_create()

bboxleft = cv2.selectROI(frame, False)
bboxright = cv2.selectROI(frame, False)

cv2.destroyAllWindows()
fvs.stop()

bbl, bbr = bboxleft ,bboxright 

In [ ]:
#for webcam , not updated

fvs = FileVideoStream(path_vid).start()
frame = fvs.read()
frame = cv2.flip(frame, 1)

trackerleft = cv2.TrackerCSRT_create()
trackerright = cv2.TrackerCSRT_create()


cv2.putText(frame, 'left select', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
bboxleft = cv2.selectROI(frame, False)

cv2.putText(frame, 'right select', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
bboxright = cv2.selectROI(frame, False)

centleft = get_centroid(bboxleft)
centright = get_centroid(bboxright)

drawc(frame, centleft)
drawc(frame, centright)

# Initialize tracker with first frame and bounding box
trackerleft.init(frame, bboxleft)
trackerright.init(frame, bboxright)


cv2.putText(frame, 'put both your hands', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);

TIMER = 3

for t in range(TIMER, 0, -1):
    cv2.putText(frame, str(t), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_BLACK, 4);  
    cv2.waitKey(1000)

cv2.destroyAllWindows()


while True:
    frame = fvs.read()
    
    if frame is None:
        break
        
    frame = cv2.flip(frame, 1)
#     frame = imutils.resize(frame, width=size_width)  

    retleft, bboxleft = trackerleft.update(frame)
    retright, bboxright = trackerright.update(frame)
    
    if not retleft == retright == True:
        print("Tracking stopped")
        break
    
    a,s  = action(bboxleft, bboxright)
    
    cv2.putText(frame, "a  "+str(pressed_a), (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
    cv2.putText(frame, "s  "+str(pressed_s), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
     
#     draw(retleft, bboxleft, frame)
#     draw(retright, bboxright, frame)

    drawc(frame, centleft)
    drawc(frame, centright)

#     write('')

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
#     cv2.putText(frame, "FPSd : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
    cv2.imshow("Tracking", frame)

    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
    
releaseAll()
    
cv2.destroyAllWindows()
fvs.stop()

In [ ]:
# Program to cancel the timer 
import threading 

def gfg(): 
	print("\nGeeksforGeeks") 

timer = threading.Timer(.1, gfg) 
timer.start() 
for i in range(3):
    print(i)
    time.sleep(1)
print("Cancelling timer\n") 
# timer.cancel() 
print("Exit\n") 


In [ ]:
def action_key(key):
    PressKey(key)
    pass

pressed_s, pressed_a = None, None
delay = 0.3
last = 0 #last extreme of steer
chng = 3 #change in last, to remove small changes fulucations
chngt = 3 #to restart change to 0, if moved towards centre by some value
prev = 0 #store last pos of steer, so that when returngin towards centre, doesnt press that key

RADIUS = 40
THRESH_ACT = 40#

def steer(bbox):
    #left
    global pressed_s, last, prev
    x, _ = get_centroid(bbox)
    diff = x - centleft[0]
    
    if abs(diff) < last or abs(prev) < abs(diff): #2ns so, new diff is more, 
        if abs(diff) < last-chng*chngt:
            last = 0
        return
    prev = diff
    last = abs(diff) + chng
    if diff > 0:
        pressed_s = D
    else:
        pressed_s = A
        
    action_key(pressed_s)
#     threading.Timer(0.3, ReleaseKey, args=[pressed_s]).start()
#     time.sleep(0.5)
#     ReleaseKey(pressed_s)
#     pressed_s = None
    return diff    
    
def accelerate(bbox):
    global pressed_a
    
    _, y = get_centroid(bbox)
    diff = - (y - centright[1])  #as y decr upward

    if abs(diff) < THRESH_ACT:
        return
    
    if diff > 0:
        pressed_a = W
    else:
        pressed_a = S
        
    action_key(pressed_a)
    
    return diff


def drawc(frame, center, radius=RADIUS, color=COLOR_BLACK, thickness=3):
    cv2.circle(frame, center, radius, color, thickness)
    
# def release():
   
    

def action(bboxleft, bboxright):
    global pressed_s, pressed_a
    
    a=accelerate(bboxright)
    s=steer(bboxleft)
#     if a is None:
    if s is None and pressed_s is not None:
        ReleaseKey(pressed_s)
        pressed_s = None
    if a is None and pressed_a is not None:
        ReleaseKey(pressed_a)
        pressed_a = None
    return a, s

def releaseAll():
    for key in [W,S,A,D]:
        ReleaseKey(key)